In [3]:
# default_exp math.func.loss

%reload_ext autoreload
%autoreload 2

# loss function | cost function | objective function
这些概念的定义有一些模糊，一般情况下：

## loss function
衡量的是在单个样本上的表现；

## cost function | objective function
cost function等同于objective function  
衡量的是在全体样本上的表现

# 回归问题常见的loss function

## SquareLoss
$$l(y,\hat y)=0.5*(y-\hat y)^2$$

### 一般用于回归问题。

举例:
对于3个样本, 

In [1]:
# export
def square_loss(y_true, y_hat):
    return 0.5 * np.power((y_true - y_hat), 2)

In [2]:
import numpy as np

In [5]:
y=np.array([0.2,0.6,0.8])
y_hat = np.array([0.18, 0.5, 0.7])

In [6]:
square_loss(y, y_hat)

array([0.0002, 0.005 , 0.005 ])

### 对应的cost function
假设它们属于一个批次，那么这个批次的mean_square_loss

In [7]:
mean_square_loss = np.sum(square_loss(y, y_hat))/len(y)
mean_square_loss

0.0034000000000000024

# 分类问题常见的loss function
分类问题的分类一般用one-hot，假设  


In [8]:
y = [(1, 0, 0), (0, 1, 0), (0, 0, 1),]
# 一般是通过softmax输出
y_hat = [(0.8, 0.1, 0.1), (0.3, 0.4, 0.3), (0.1, 0.4, 0.5)]

如何来定义loss？是否可以用SquareLoss？如果非要使用SquareLoss的话，有两种方案：
1. 把y使用np.argmax还原成数值编码[0, 1, 2]  
对于上述而言例子而言，loss结果是0，完美  

但是假设y_hat的数值编码是[0, 1, 0]和另一种[0, 1, 1]的loss应该是一样的(都预测错1个)，但是使用SquareLoss的话，[0, 1, 0]的loss大于[0, 1, 1]的loss。所以，这种方案不合适

1. 第二种方案是
如计算第一个样本的$loss=0.5*((1-0.8)^2+(0-0.1)^2+(0-0.1)^2)/3$  
这样做有显得太苛刻了(__from李沐__)，必须要使得y_hat和y完全相同，loss才等于0。而实际上，例子中的结果的loss就应该是0！

__综上，使用SquareLoss作为分类问题的loss并不合适！__

## LogLoss
https://medium.com/datadriveninvestor/understanding-the-log-loss-function-of-xgboost-8842e99d975d

Log loss, short for logarithmic loss is a loss function for classification that quantifies the price paid for the inaccuracy of predictions in classification problems. Log loss penalizes false classifications by taking into account the probability of classification. 
### LogLossBinary

主要用作2分类问题的损失函数，如作为逻辑回归的loss function。
$$l(y,\hat y)=-yln(\hat y)-(1-y)ln(1-\hat y)$$

其中$y和\hat y都是标量。\hat y$表示预测为1的概率，那么$1-\hat y$就是预测为0的概率。

因为y的取值只有0或1，那么实际中表示式的两部分只有一部分会生效：当y=1时，前半部分生效;当y=0时，后半部分生效

上式可以拆解为条件表达式：

In [10]:
def log_loss_cond(actual, predict_prob):
    if actual == 1:  
        # use natural logarithm
        return -log(predict_prob) 
    else:
        return -log(1 - predict_prob)

In [11]:
def log_loss(y, p):
    # Avoid division by zero
    p = np.clip(p, 1e-15, 1 - 1e-15)
    return - y * np.log(p) - (1 - y) * np.log(1 - p)

In [12]:
log_loss(1, 0.5)

0.6931471805599453

In [13]:
log_loss(0, 0.5)

0.6931471805599453

### CrossEntropyLoss(交叉熵损失)
Cross-entropy is the more generic form of logarithmic loss when it comes to machine learning algorithms. 

While log loss is used for binary classification algorithms, cross-entropy serves the same purpose for multiclass classification problems. In other words, log loss is used when there are 2 possible outcomes and cross-entropy is used when there are more than 2 possible outcomes.

主要用作多分类问题的损失函数。  

logistic变换，其中x为标量
$$sigmoid(x)=\frac{1}{1+exp(x)}$$
扩展到多分类的
$$softmax(\vec x)=\frac{exp(\vec x)}{\sum exp(\vec x)}$$
LogLoss对应到多分类的结果为CrossEntropyLoss  
$$l(\vec y, \vec{\hat y})=-\sum_{i=1}^n{y_iln(\hat y_i)}$$

理解：因为采用one-hot, $\vec y$中只有一个为1，
* 假设$y_i=1$，如果对应的预测值$\hat y_i$为1，那么loss值为0；
* 如果对应的预测值$\hat y_i$接近0，那么loss值接近正无穷大

In [1]:
def cross_entropy_loss(y_true, y_hat):
    """
    multi分类loss function. 每一个样本的预测值用一个one-hot vector表示
    :param y_true: np.array([(1, 0, 0), (1, 0, 0), (1, 0, 0)])
    :param y_hat: np.array([(0.18, 0.5, 0.7), (0.18, 0.5, 0.7),(0.18, 0.5, 0.7),])
    :return:
    """
    # Avoid division by zero
    y_hat = np.clip(y_hat, 1e-15, 1 - 1e-15)
    # 注意计算是由于-y_true * np.log(y_hat) element-wise，
    # 每一行只有一个值非0，所以按照cross_entropy_loss有两个\sum，可以合并成1个
    return np.sum(-y_true * np.log(y_hat))/len(y_true)

In [3]:
import numpy as np

In [7]:
y_true = np.array([(1, 0, 0), (1, 0, 0), (1, 0, 0)])
y_hat = np.array([(0.48, 0.51, 0.01), (0.48, 0.51, 0.01),(0.48, 0.51, 0.01),])

In [8]:
cross_entropy_loss(y_true, y_hat)

0.7339691750802005

# nb_export

In [4]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted engineering_nbdev.ipynb.
Converted index.ipynb.


In [7]:
!nbdev_build_docs

No notebooks were modified
converting /Users/luoyonggui/PycharmProjects/nbdevlib/index.ipynb to README.md
